![title](img/ESPN.png)

In [1]:
%run ./PProcess0.ipynb
driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\Valentin\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


====== WebDriver manager ======
Current google-chrome version is 111.0.5563
Get LATEST driver version for 111.0.5563
Driver [C:\Users\Valentin\.wdm\drivers\chromedriver\win32\111.0.5563.64\chromedriver.exe] found in cache


In [4]:
LastGames = 10
History.drop_duplicates(inplace=True)
groups = History.groupby('Home')
h = pd.concat([g.iloc[:LastGames] for _, g in groups if len(g) > LastGames])
#h = h[pd.to_datetime(h['Date']) > pd.Timestamp('2023-01-01')]

score_ranges = list(range(0, 7))

# Loop over the score ranges and count the number of scores greater than and less than each range
for score_range in tqdm(score_ranges):
    #h[f'H>{score_range}'] = (h['ScoreH'] > score_range).astype(int)
    #h[f'A>{score_range}'] = (h['ScoreA'] > score_range).astype(int)
    #h[f'H<{score_range}'] = (h['ScoreH'] < score_range).astype(int)
    #h[f'A<{score_range}'] = (h['ScoreA'] < score_range).astype(int)
    h[f'B>{score_range}'] = (h['But'] > score_range).astype(int)
    h[f'B<{score_range}'] = (h['But'] < score_range).astype(int)
    
    h[f'DHB>{score_range}'] = ((h['But'] > score_range) & (h['DH'] == 1)).astype(int)
    h[f'DHB<{score_range}'] = ((h['But'] < score_range) & (h['DH'] == 1)).astype(int)
    
    h[f'HAB>{score_range}'] = ((h['But'] > score_range) & (h['HA'] == 1)).astype(int)
    h[f'HAB<{score_range}'] = ((h['But'] < score_range) & (h['HA'] == 1)).astype(int)
    
    h[f'DAB>{score_range}'] = ((h['But'] > score_range) & (h['DA'] == 1)).astype(int)
    h[f'DAB<{score_range}'] = ((h['But'] < score_range) & (h['DA'] == 1)).astype(int)
    
    h['n'] = 1
    
HomeStats = h.groupby('Home')[
                              [f'B>{score_range}' for score_range in score_ranges] +
                              [f'B<{score_range}' for score_range in score_ranges] +
                              [f'DHB>{score_range}' for score_range in score_ranges] +
                              [f'DHB<{score_range}' for score_range in score_ranges] +
                              [f'HAB>{score_range}' for score_range in score_ranges] +
                              [f'HAB<{score_range}' for score_range in score_ranges] +
                              [f'DAB>{score_range}' for score_range in score_ranges] +
                              [f'DAB<{score_range}' for score_range in score_ranges] +
                              ['Ds'] +
                              ['Hs'] +
                              ['As'] +
                              ['DA'] +
                              ['DH'] +
                              ['HA'] +
                              ['LDEMs'] +
                              ['LDEMPs'] +
                              ['DALDEMs'] +
                              ['DHLDEMs'] +
                              ['HALDEMs'] +
                              ['DALDEMPs'] +
                              ['DHLDEMPs'] +
                              ['HALDEMPs'] +
                              ['n']].sum()

HomeStats['Home'] = HomeStats.index
#HomeStats.reset_index(drop=True)
#HomeStats.index.name = None
HomeStats = HomeStats[HomeStats["n"]>=6]
HomeStats = HomeStats.set_index(['Home']).add_prefix('1').reset_index()

AwayStats = h.groupby('Away')[[#f'H>{score_range}' for score_range in score_ranges] +
                              #[f'A>{score_range}' for score_range in score_ranges] +
                              #[f'H<{score_range}' for score_range in score_ranges] +
                              #[f'A<{score_range}' for score_range in score_ranges] +
                               f'B>{score_range}' for score_range in score_ranges] +
                              [f'B<{score_range}' for score_range in score_ranges] +
                              [f'DHB>{score_range}' for score_range in score_ranges] +
                              [f'DHB<{score_range}' for score_range in score_ranges] +
                              [f'HAB>{score_range}' for score_range in score_ranges] +
                              [f'HAB<{score_range}' for score_range in score_ranges] +
                              [f'DAB>{score_range}' for score_range in score_ranges] +
                              [f'DAB<{score_range}' for score_range in score_ranges] +
                              ['Ds'] +
                              ['Hs'] +
                              ['As'] +
                              ['DA'] +
                              ['DH'] +
                              ['HA'] +
                              ['LDEMs'] +
                              ['LDEMPs'] +
                              ['DALDEMs'] +
                              ['DHLDEMs'] +
                              ['HALDEMs'] +
                              ['DALDEMPs'] +
                              ['DHLDEMPs'] +
                              ['HALDEMPs'] +
                              ['n']].sum()

AwayStats['Away'] = AwayStats.index
#AwayStats.reset_index(drop=True)
#AwayStats.index.name = None
AwayStats = AwayStats[AwayStats["n"]>=6]
AwayStats = AwayStats.set_index(['Away']).add_prefix('2').reset_index()

100%|██████████| 7/7 [00:00<00:00, 142.73it/s]


In [13]:
from datetime import datetime
HistoryHA = pd.DataFrame(Hist)
HistoryHA["Date"] = HistoryHA["Date"].apply(lambda x: datetime.strptime(x,  "%a, %d %b %Y").date())
HistoryHA = HistoryHA.sort_values(by=["Team", "Date"], ascending=False)
HistoryHA[['ScoreH', 'ScoreA']] = HistoryHA['Score'].str.split(' - ', expand=True).astype(int)
HistoryHA[['But']] = HistoryHA['ScoreH'] + HistoryHA['ScoreA']
HistoryHA['Result'] = np.where(HistoryHA['ScoreH'] == HistoryHA['ScoreA'], 'D',
                             np.where(np.logical_and(HistoryHA['ScoreH'] > HistoryHA['ScoreA'], HistoryHA['Team'] == HistoryHA['Home']), 'W',
                                      np.where(np.logical_and(HistoryHA['ScoreH'] < HistoryHA['ScoreA'], HistoryHA['Team'] == HistoryHA['Away']), 'W',
                                      'L')))
HistoryHA = HistoryHA.reset_index(drop=True)

team = ""
teamChange = ""
stop_flags = {
    'W': True,
    'L': True,
    'D': True,
    'WD': True,
    'LD': True,
    'WL': True,
    
    'HW': True,
    'HL': True,
    'HD': True,
    'HWD': True,
    'HLD': True,
    'HWL': True,
    
    'AW': True,
    'AL': True,
    'AD': True,
    'AWD': True,
    'ALD': True,
    'AWL': True
}
counts = {
    'W': 0,
    'L': 0,
    'D': 0,
    'WD': 0,
    'LD': 0,
    'WL': 0,
    
    'HW': 0,
    'HL': 0,
    'HD': 0,
    'HWD': 0,
    'HLD': 0,
    'HWL': 0,
    
    'AW': 0,
    'AL': 0,
    'AD': 0,
    'AWD': 0,
    'ALD': 0,
    'AWL': 0
}
# HistoryHA = HistoryHA[(HistoryHA["Team"] == "Querétaro") | (HistoryHA["Team"] == "América")].reset_index(drop=True)
# HistoryHA = HistoryHA[(HistoryHA["Team"] == "Newcastle United") | (HistoryHA["Team"] == "Fulham")].reset_index(drop=True)
# HistoryHA = HistoryHA[(HistoryHA["Team"] == "Newcastle United")].reset_index(drop=True)
# HistoryHA = HistoryHA[(HistoryHA["Team"] == "Cremonese")].reset_index(drop=True)
# HistoryHA = HistoryHA[(HistoryHA["Team"] == "Cremonese") | (HistoryHA["Team"] == "AC Milan") ].reset_index(drop=True)

for index, row in HistoryHA.iterrows():
    result_mapping = {
        "L": ("LD", "WL"),
        "D": ("WD", "LD"),
        "W": ("WD", "WL"),
    }
    double1, double2 = result_mapping.get(row['Result'])
    
    w = ["" ,"H" if row['Team'] == row['Home'] else "A"]
    for where in w:
        if (where == "H" and row["Team"] == row["Home"]) or (where == "A" and row["Team"] == row["Away"]) or (where == ""):
            allRes = [row['Result'], double1, double2]
            if team != row['Team']:
                team = row['Team']
                if (teamChange != row['Team']):
                    if teamChange == "":
                        teamChange = row['Team']
                    else:
                        teamChange = row['Team']
                        for result in ['W', 'L', 'D', 'WD', 'LD', 'WL']:
                            if stop_flags[where+result]:
                                HistoryHA.at[index-1, where+result] = counts[where+result]
                stop_flags = {key: True for key in stop_flags}
                counts = {key: 0 for key in counts}
                
            for result in ['W', 'L', 'D', 'WD', 'LD', 'WL']:
                if stop_flags[where+result]:
                    if (result in allRes):
                        counts[where+result] += 1
                
                    else:
                        HistoryHA.at[index, where+result] = counts[where+result]
                        stop_flags[where+result] = False
                        
#             correct last record of dataset
            if (index == (len(HistoryHA)-1)):
                if teamChange == "":
                    teamChange = row['Team']
                else:
                    teamChange = row['Team']
                    for result in ['W', 'L', 'D', 'WD', 'LD', 'WL']:
                        if where != "":
                            wadd = "H" if (where == "A") and (len(where) == 1) else "A"
                            HistoryHA.at[index-1, wadd+result] = counts[wadd+result]
                        if stop_flags[where+result]:
                            HistoryHA.at[index-1, where+result] = counts[where+result]
                                
HistoryHA.fillna(0, inplace=True)
# HistoryHA[HistoryHA["Team"] == "Borussia Dortmund"].head(15)
# HistoryHA[HistoryHA["Team"] == "Fortuna Düsseldorf"].head(20)

Streak = HistoryHA.groupby('Team')[['W','L','D','WD','LD','WL',
                                        'HW','HL','HD','HWD','HLD','HWL',
                                        'AW','AL','AD','AWD','ALD','AWL']].max()
Streak['max'] = Streak.iloc[:, 0:6].max(axis=1)
Streak['maxH'] = Streak.iloc[:, 6:12].max(axis=1)
Streak['maxA'] = Streak.iloc[:, 12:18].max(axis=1)

mini = 10
Streak = Streak.sort_values(['max', 'maxH'], ascending=False)
# Streak = Streak[(Streak["maxH"] >= mini) | (Streak["maxA"] >= mini)]#.head(20)
Streak['Team'] = Streak.index
Streak = Streak.set_index(['Team']).reset_index()
Streak.style.background_gradient(cmap="viridis").set_precision(0)

In [15]:
# Newcastle United
# HistoryHA[(HistoryHA["Team"] == "Newcastle United")]
HistoryHA